<a href="https://colab.research.google.com/github/breathofthe/predicting-pm10/blob/main/predecting_pm10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pm10 값 예측하기

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [8]:
data = pd.read_csv('PM10_fin_2.csv', parse_dates=['Date'], index_col='Date')

##데이터 전처리

첫번째 열이 날짜데이터이므로 `pm10` 예측에는 사용되지 않을 것이다.

In [3]:
data.head()

,TRUCK,VESSEL_NO,VESSEL_GT,PM10,O3,NO2,CO,SO2,TEMP,RAIN,WIND,HMD
Date,,,,,,,,,,,,
2020-09-01 07:00:00,2.0,68,934538.0,17.0,0.025,0.009,0.1,0.001,24.0,0.0,2.7,70.0
2020-09-01 08:00:00,91.0,71,965712.0,17.0,0.020,0.014,0.1,0.001,24.6,0.0,3.4,69.0
2020-09-01 09:00:00,145.0,75,1111570.0,14.0,0.019,0.015,0.1,0.002,24.7,0.0,2.0,72.0
2020-09-01 10:00:00,153.0,77,1120758.0,21.0,0.021,0.014,0.1,0.002,26.9,0.0,2.4,67.0
2020-09-01 11:00:00,178.0,72,1113430.0,13.0,0.026,0.012,0.2,0.002,28.4,0.0,3.0,63.0


**훈련데이터와 시험데이터 분리**

`2020-09-01`부터`2021-09-30`기간에 데이터를 훈련데이터로

`2021-10-01`부터`2021-12-31`기간에 데이터를 시험데이터로 하자

In [9]:
train_data = data['2020-09-01':'2021-09-30']
test_data = data['2021-10-01':'2021-12-31']

`feature`들이 서로 다른 범위를 가지므로
`scailng`하여 일정한 범위로 조절 해야한다.

> 이때 첫번째 열은 제외하고 `scaling`을 진행한다

In [10]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data.iloc[:, 1:])
test_scaled = scaler.transform(test_data.iloc[:, 1:])

##시계열 데이터셋 함수 설정

특정시점의 `pm10`을 예측하기 위해서는 3일동안 측정된 데이터가 필요하다.

즉, 특정시점으로부터 직전 3일동안의 `pm10`의 값이 필요하다.

그런데 우리는 예측하고자하는 특정시점의 `pm10`값을 모를뿐 그 시점에서 다른 변수들을 관측이 가능하다.

따라서 `pm10`변수를 제외한 나머지 `feature`들에 대해서는 현재시점을 포함한 4일동안의 관측된 데이터가 필요하다.



In [11]:

def create_dataset(data, time_step_X=4, time_step_Y=3):
    X, Y = [], []
    for i in range(len(data) - max(time_step_X, time_step_Y)):
        # 나머지 변수에 대해 4일 동안의 데이터를 X에 추가
        X.append(data[i:(i + time_step_X)])
        # PM10 변수에 대해 3일 동안의 데이터를 Y에 추가
        Y.append(data[i + time_step_Y, 3])  # PM10은 네 번째 열 (인덱스 3)
    return np.array(X), np.array(Y)

- time_step을 `pm10`과 나머지 변수에 대해 각각 다르게 설정

- 앞서 만든 함수를 이용하여 시계열 데이터 구성

- lstm 모델에 학습이 가능한 형태로 데이터 재구성

In [16]:
time_step_Y=72  # 3일간의 데이터
time_step_X=96  # 4일간의 데이터
X_train, y_train = create_dataset(train_scaled, time_step_X, time_step_Y)
X_test, y_test = create_dataset(test_scaled, time_step_X , time_step_Y)

X_train = X_train.reshape(X_train.shape[0], time_step_X, X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], time_step_X, X_test.shape[2])

**모델생성**

In [18]:
# LSTM 모델 생성
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step_X, X_train.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

**모델학습**

In [19]:
model.fit(X_train, y_train, batch_size=16, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
508/508 [==============================] - 50s 91ms/step - loss: 0.0086 - val_loss: 0.0052
Epoch 2/20
508/508 [==============================] - 42s 83ms/step - loss: 0.0043 - val_loss: 0.0038
Epoch 3/20
508/508 [==============================] - 42s 83ms/step - loss: 0.0037 - val_loss: 0.0036
Epoch 4/20
508/508 [==============================] - 42s 83ms/step - loss: 0.0034 - val_loss: 0.0028
Epoch 5/20
508/508 [==============================] - 42s 83ms/step - loss: 0.0031 - val_loss: 0.0024
Epoch 6/20
508/508 [==============================] - 47s 93ms/step - loss: 0.0027 - val_loss: 0.0034
Epoch 7/20
508/508 [==============================] - 50s 98ms/step - loss: 0.0027 - val_loss: 0.0025
Epoch 8/20
508/508 [==============================] - 55s 109ms/step - loss: 0.0024 - val_loss: 0.0019
Epoch 9/20
508/508 [==============================] - 45s 88ms/step - loss: 0.0023 - val_loss: 0.0016
Epoch 10/20
508/508 [==============================] - 48s 94ms/step - loss: 0.00

**성능평가**

In [20]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# 성능 평가
mae = mean_absolute_error(y_test, test_predict)
mse = mean_squared_error(y_test, test_predict)
rmse = np.sqrt(mse)

print(f'MAE: {mae}, MSE: {mse}, RMSE: {rmse}')

61/61 [==============================] - 1s 24ms/step
MAE: 0.028255972392607697, MSE: 0.0014522786130558543, RMSE: 0.0381087734394044
